In [ ]:
import sys
sys.path.append("/ccs/proj/csc308/klion/python_lib")

In [ ]:
import numpy as np
import yt
import matplotlib.pyplot as plt
import consts as c
import si_consts as s
import load_fonts as lf

In [ ]:
ds = yt.load('/gpfs/alpine/proj-shared/csc308/klion/recon_eqbm/diags/diags00001/')

In [ ]:
ad0 = ds.covering_grid(level=0, left_edge=ds.domain_left_edge, dims=ds.domain_dimensions)

ad = ds.all_data()
rho = ad0['boxlib', 'rho'].to_ndarray()[:,:,0]
rhoe = ad0['boxlib', 'rho_electrons'].to_ndarray()[:,:,0]
jy = ad0['boxlib', 'jy'].to_ndarray()[:,:,0]
Bz = ad0['boxlib', 'Bz'].to_ndarray()[:,:,0]
x = ad0['boxlib', 'x'].to_ndarray()[:,:,0]
y = ad0['boxlib', 'y'].to_ndarray()[:,:,0]
x_line = x[:,0]
y_line = y[0,:]
dx = x_line[1] - x_line[0]
dy = y_line[1] - y_line[0]
x_edges = np.zeros(len(x_line)+1)
y_edges = np.zeros(len(y_line)+1)
x_edges[1:] = x_line + dx / 2
y_edges[1:] = y_line + dy / 2
x_edges[0] = x_line[0] - dx / 2
y_edges[0] = y_line[0] - dy / 2
xcs = x_edges[-1]/2
y_quar = y_edges[-1]/2
thresh = dx*4
nx = len(x_line)
ny = len(y_line)

In [ ]:
def center(pfilter, data):
    filter = np.abs(data[(pfilter.filtered_type, "particle_position_y")]) < thresh
    return filter

yt.add_particle_filter(
    "center", function=center, filtered_type="all", requires=["particle_position_y"]
)
ds.add_particle_filter("center")

def top_half(pfilter, data):
    filter = np.abs(data[(pfilter.filtered_type, "particle_position_y")] - y_quar) < thresh
    return filter

yt.add_particle_filter(
    "top_half", function=center, filtered_type="all", requires=["particle_position_y"]
)
ds.add_particle_filter("top_half")

def bottom_half(pfilter, data):
    filter = np.abs(data[(pfilter.filtered_type, "particle_position_y")] + y_quar) < thresh
    return filter

yt.add_particle_filter(
    "bottom_half", function=center, filtered_type="all", requires=["particle_position_y"]
)
ds.add_particle_filter("bottom_half")

In [ ]:
eta = 200
B0 = 0.4175177341
nbe = 5.67917e16
nde = nbe * 5.
lambda_e = s.c / s.e * (s.me * s.e0 / nde) ** 0.5
theor_tot = B0 ** 2 / s.mu0 * (4 + eta) / (2 * eta)

In [ ]:
particle_set = 'center'
e_px = ad[particle_set, 'particle_momentum_x'].to_ndarray()
e_py = ad[particle_set, 'particle_momentum_y'].to_ndarray()
e_pz = ad[particle_set, 'particle_momentum_z'].to_ndarray()
e_x = ad[particle_set, 'particle_position_x'].to_ndarray()
e_y = ad[particle_set, 'particle_position_y'].to_ndarray()
e_w = ad[particle_set, 'particle_weight'].to_ndarray()

g = np.sqrt(1 + (e_px ** 2 + e_py ** 2 + e_pz ** 2)/(s.me*s.c)**2)
betax = e_px / s.me / s.c / g
part_pres = s.me * e_w * betax ** 2 * g * s.c ** 2

i_x = np.floor((e_x - x_edges[0]) / dx)
i_y = np.floor((e_y - y_edges[0]) / dy)

d_vol = dx * (max(e_y) - min(e_y))

#### Pressure Balance Verification

In [ ]:
pres = np.zeros(len(x_line))
for i_p in range(len(e_px)):
    pres[int(i_x[i_p])] += part_pres[i_p] / d_vol

In [ ]:
plt.plot(x_line, 1e-4 * pres, label="thermal")
plt.plot(x_line, 1e-4 * Bz[:,ny//2]**2 / s.mu0/2, label="magnetic")
plt.plot(x_line, 1e-4 * (pres + Bz[:,ny//2]**2 / s.mu0/2), label="total")
plt.plot(x_line, 1e-4 * np.ones_like(x_line) * theor_tot, color='k', linestyle='--', label="theoretical")
plt.xlim(xcs-5 * 0.05, xcs +5 * 0.05) 
plt.legend()
plt.xlabel("x [m]")
plt.ylabel("pressure [10$^4$ J/m$^3$]")
#plt.savefig("warpx_figs/pressure_balance_big_zoom_{}.pdf".format(particle_set))
#plt.clf()

In [ ]:
plt.plot(x_line, 1e-4 * pres, label="thermal")
plt.plot(x_line, 1e-4 * Bz[:,ny//2]**2 / s.mu0/2, label="magnetic")
plt.plot(x_line, 1e-4 * (pres + Bz[:,ny//2]**2 / s.mu0/2), label="total")
plt.plot(x_line, 1e-4 * np.ones_like(x_line) * theor_tot, color='k', linestyle='--', label="theoretical")
plt.xlim(-xcs-5 * 0.05, -xcs +5 * 0.05) 
plt.legend()
plt.xlabel("x [m]")
plt.ylabel("pressure [10$^4$ J/m$^3$]")

In [ ]:
pres_bin = (pres[::4] + pres[1::4] + pres[2::4] + pres[3::4])/4.
x_bin = (x_line[::4] + x_line[1::4] + x_line[2::4] + x_line[3::4])/4.
Bz_bin = (Bz[::4,ny//4]**2 + Bz[1::4,ny//4]**2 + Bz[2::4,ny//4]**2 + Bz[3::4,ny//4]**2)/4.
            
plt.plot(x_line, (pres + Bz[:,ny//2]**2 / s.mu0 / 2 -theor_tot) / theor_tot)
plt.xlim(xcs-5 * 0.05, xcs +5 * 0.05)
plt.ylabel("Fractional error in total pressure")
plt.xlabel("x [m]")
#plt.savefig("warpx_figs/pressure_balance_big_frac_zoom_{}.pdf".format(particle_set))
#plt.show()

In [ ]:
pres_bin = (pres[::4] + pres[1::4] + pres[2::4] + pres[3::4])/4.
x_bin = (x_line[::4] + x_line[1::4] + x_line[2::4] + x_line[3::4])/4.
Bz_bin = (Bz[::4,ny//4]**2 + Bz[1::4,ny//4]**2 + Bz[2::4,ny//4]**2 + Bz[3::4,ny//4]**2)/4.
            
plt.plot(x_line, (pres + Bz[:,ny//2]**2 / s.mu0 / 2 -theor_tot) / theor_tot)
plt.xlim(-xcs-5 * 0.05, -xcs +5 * 0.05)
plt.ylabel("Fractional error in total pressure")
plt.xlabel("x [m]")
#plt.savefig("warpx_figs/pressure_balance_big_frac_zoom_{}.pdf".format(particle_set))
#plt.show()

In [ ]:
plt.semilogy(x_line, np.abs((pres + Bz[:,ny//2]**2 / s.mu0 / 2 -theor_tot) / theor_tot))
plt.ylabel("Abs fractional error in total pressure")
plt.xlabel("x [m]")
plt.savefig("warpx_figs/pressure_balance_big_absfrac_{}.pdf".format(particle_set))

#### Ampere's Law Verification

In [ ]:
dBz_dx = np.diff(Bz, axis=0) / np.diff(x, axis=0)

In [ ]:
plt.plot(x_line[1:],s.mu0*jy[1:,ny//2])
plt.plot(x_line[1:],-dBz_dx[:,ny//2])

In [ ]:
plt.plot(x_line[1:],s.mu0*jy[1:,ny//2])
plt.plot(x_line[1:],-dBz_dx[:,ny//2])
plt.xlim(-xcs-5 * 0.05, -xcs +5 * 0.05)
plt.ylim(-1,8)

In [ ]:
plt.plot(x_line[1:],s.mu0*jy[1:,ny//2])
plt.plot(x_line[1:],-dBz_dx[:,ny//2])
plt.xlim(xcs-5 * 0.05, xcs +5 * 0.05)
plt.ylim(-8,1)

In [ ]:
plt.plot(x_line[1:],(s.mu0*jy[1:]+dBz_dx)[:,ny//2]/(B0/(0.05)))
plt.xlim(xcs-0.05*5, xcs+0.05*5)
plt.axvline(xcs-0.05*2, c='orange')
plt.axvline(xcs+0.05*2, c='orange')
plt.axvline(xcs, c='orange', linestyle='--')
plt.ylabel("$(J_y + \partial_x B_z)/(B_0/\delta)$")
plt.ylim(-2e-1, 2e-1)
plt.xlabel("x [m]")
#plt.savefig("warpx_figs/amperes_frac_1.pdf")

In [ ]:
plt.plot(x_line[1:],(s.mu0*jy[1:]+dBz_dx)[:,ny//2]/(B0/(0.05)))
plt.xlim(-xcs-0.05*5, -xcs+0.05*5)
plt.axvline(-xcs-0.05*2, c='orange')
plt.axvline(-xcs+0.05*2, c='orange')
plt.axvline(-xcs, c='orange', linestyle='--')
plt.ylabel("$(J_y + \partial_x B_z)/(B_0/\delta)$")
plt.ylim(-2e-1, 2e-1)
plt.xlabel("x [m]")

In [ ]:
plt.plot(x_line[1:],(s.mu0*jy[1:]+dBz_dx)[:,ny//2]/dBz_dx[:,ny//2])
plt.xlim(xcs-0.05*5, xcs+0.05*5)
plt.axvline(xcs-0.05*2, c='orange')
plt.axvline(xcs+0.05*2, c='orange')
plt.axvline(xcs, c='orange', linestyle='--')
plt.ylim(-2e-1, 2e-1)
plt.xlabel("x [m]")
plt.ylabel("$(J_y + \partial_x B_z)/\partial_x B_z$")

In [ ]:
plt.plot(x_line[1:],(s.mu0*jy[1:]+dBz_dx)[:,ny//2]/dBz_dx[:,ny//2])
plt.xlim(-xcs-0.05*5, -xcs+0.05*5)
plt.axvline(-xcs-0.05*2, c='orange')
plt.axvline(-xcs+0.05*2, c='orange')
plt.axvline(-xcs, c='orange', linestyle='--')
plt.ylim(-2e-1, 2e-1)
plt.xlabel("x [m]")
plt.ylabel("$(J_y + \partial_x B_z)/\partial_x B_z$")
#plt.savefig("warpx_figs/amperes_frac_2.pdf")